In [5]:
import os
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
from langchain.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://en.wikipedia.org/wiki/David_Lynch")

In [6]:
docs = loader.load()

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [82]:
chunk_size =500
chunk_overlap = 100

In [83]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separators=["\n\n", "\n", " ", ""],
    length_function = len
)

In [84]:
chunks = r_splitter.split_documents(docs)

In [85]:
len(docs)

1

In [86]:
len(chunks)

447

In [8]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [9]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Provide your Google API key here")

Provide your Google API key here ········


In [10]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

In [12]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone

pc = Pinecone(api_key="")
index = pc.Index("wikrag")

vector_store = PineconeVectorStore(embedding=embeddings, index=index)

In [108]:
document_ids = vector_store.add_documents(documents=chunks)

In [13]:
question = "where was david lynch born?"
results = vector_store.similarity_search(
    question,
    k=2,
)
for res in results:
    print(f"* {res.page_content}")

* David Keith Lynch was born in Missoula, Montana, on January 20, 1946.[7]: 1  The first film he saw was Henry King's Wait till the Sun Shines, Nellie (1952).[8] His father, Donald Walton Lynch (1915–2007), was a research scientist working for the U.S. Department of Agriculture (USDA), and his mother, Edwina "Sunny" Lynch (née Sundholm; 1919–2004), was an English language tutor. Two of Lynch's maternal great-grandparents were Finnish-Swedish immigrants who arrived in the U.S. during the 19th
* Retrieved from "https://en.wikipedia.org/w/index.php?title=David_Lynch&oldid=1281640660"


In [15]:
if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")


Enter API key for Groq:  ········


In [16]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("llama3-8b-8192", model_provider="groq")

In [17]:
from langchain.chains import RetrievalQA

In [18]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vector_store.as_retriever()
)

In [ ]:
result = qa_chain({"query":question})

In [ ]:
result["result"]

In [26]:
question = "did david win oscar?"

In [27]:
result = qa_chain({"query":question})

In [28]:
result["result"]

'According to the provided context, David Lynch has not won an Academy Award (Oscar) directly. However, he has received an Academy Honorary Award in 2025, which is a non-competitive award given for his contributions to the film industry.'